# Chicago City Data Socrata API
  
Author: Mourad Askar

**Download the data from Chicago Data Portal**  


In [9]:
from datetime import date
from platform import python_version

import IPython
import IPython.display

import pandas as pd

from sodapy import Socrata


%precision 2
pd.set_option('display.max_columns',200)
pd.set_option('display.precision',2)
pd.set_option('display.float_format','{:,.4f}'.format)

print('python',python_version())
print(pd.__name__, pd.__version__)

python 3.9.10
pandas 1.4.0


## Download Dataset

### Socrata API batch downloader

In [10]:
# Code to download the dataset in batches to adhere with socrata api 50K data limit
def download_from_chicago_data(socrata_api_domain, app_token, dataset_identifier, **query_params):
    # Get dateset metadata
    # References:
    #  https://pypi.org/project/sodapy/
    #  https://dev.socrata.com/docs/transforms/

    content_type = 'csv' # ['json', 'csv']
    df_raw = None
    # Records limit per batch (Max = 50,000)
    limit = 50000
    # Init offset to zero
    offset = 0

    print(f'offset: {offset}')
    print('Batch Records:', 0)
    print('Total Records:', 0)
    print('\nloading...')

    with Socrata(socrata_api_domain, app_token, timeout=100) as client:
        while(True):            
            data = client.get(
                dataset_identifier, content_type,
                **query_params,
                limit = limit,
                offset = offset
                )

            df_raw_batch = pd.DataFrame(data) # convert to pandas dataframe
            df_raw_batch = df_raw_batch.rename(columns=df_raw_batch.iloc[0]).drop(df_raw_batch.index[0]) # rename df columns as first row and drop it

            if offset == 0:
                df_raw = df_raw_batch.copy()
            else:
                df_raw = df_raw.append(df_raw_batch)

            IPython.display.clear_output(wait=True)
            print(f'offset: {offset}')
            print('Batch Records:', df_raw_batch.shape[0])
            print('Total Records:', df_raw.shape[0])
            print('\nloading...')
                    
            if df_raw_batch.shape[0] < limit:
                # Break if returned batch is less than limit size
                break
            else:
                # Else, offset by batch limit size, and loop again
                offset += limit
        
    IPython.display.clear_output(wait=True)

    print('Total Records:', df_raw.shape[0])
    print()
    return df_raw

### Set Dataset Parameters

In [11]:
# Load API key from file
with open('/Users/maskar/.keys/.chicago_data', 'rt', ) as token_file:
        app_token = token_file.readline().strip()
        #print(app_token)

In [12]:
# Set these to the appropriate values
dataset_name = 'chicago-divvy-trips-data' # custom dataset name for file name
dataset_identifier = 'fg6s-gzvg' # from https://data.cityofchicago.org/Transportation/Divvy-Trips/fg6s-gzvg

In [13]:
# Dont't need to change these
date_stamp = date.today().strftime('%Y%m%d')
print(f'date_stamp: {date_stamp}')

file_output_csv = f'{date_stamp}_{dataset_name}.csv'
file_output_parquet = f'{date_stamp}_{dataset_name}.parquet'

socrata_api_domain = 'data.cityofchicago.org'

date_stamp: 20220205


In [14]:
# Get dateset metadata
# Reference: https://pypi.org/project/sodapy/
with Socrata(socrata_api_domain, app_token) as client:
    meta_data = client.get_metadata(dataset_identifier)
    #print(meta_data['columns'])
    [print(f"{column['name']} ({column['dataTypeName']})") for column in meta_data['columns']]
    # Expect to query column names with _ instead of spaces

TRIP ID (number)
START TIME (calendar_date)
STOP TIME (calendar_date)
BIKE ID (text)
TRIP DURATION (number)
FROM STATION ID (text)
FROM STATION NAME (text)
TO STATION ID (text)
TO STATION NAME (text)
USER TYPE (text)
GENDER (text)
BIRTH YEAR (number)
FROM LATITUDE (number)
FROM LONGITUDE (number)
FROM LOCATION (point)
TO LATITUDE (number)
TO LONGITUDE (number)
TO LOCATION (point)
Boundaries - ZIP Codes (number)
Zip Codes (number)
Community Areas (number)
Wards (number)


### Query Parameters

In [15]:
# Limit date range
date_from = '2019-01-01'
date_to   = '2019-01-02'

### Query Socrata API

In [16]:
df_raw = download_from_chicago_data(socrata_api_domain, app_token, dataset_identifier,
                select = "TRIP_ID, START_TIME, BIKE_ID",
                where = f"date_trunc_ymd(START_TIME) between '{date_from}' and '{date_to}'", # date_trunc_ymd sets the time component to 00:00:00
                order = "START_TIME DESC",
                )

Total Records: 5482



### Review Result Dataset

In [17]:
# Show data range downloaded
print(df_raw.describe(datetime_is_numeric=True).T)
print()

# Show data info
print(df_raw.info())
print()

display(df_raw)

           count unique                      top freq
TRIP_ID     5482   5482                 21749149    1
START_TIME  5482   5283  2019-01-02T08:26:27.000    4
BIKE_ID     5482   1944                     6118   16

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5482 entries, 1 to 5482
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   TRIP_ID     5482 non-null   object
 1   START_TIME  5482 non-null   object
 2   BIKE_ID     5482 non-null   object
dtypes: object(3)
memory usage: 128.6+ KB
None



,TRIP_ID,START_TIME,BIKE_ID
1,21749149,2019-01-02T23:59:55.000,896
2,21749148,2019-01-02T23:53:05.000,3876
3,21749147,2019-01-02T23:50:06.000,3384
4,21749146,2019-01-02T23:48:39.000,2454
5,21749145,2019-01-02T23:44:56.000,6397
...,...,...,...
5478,21742447,2019-01-01T00:14:52.000,1170
5479,21742446,2019-01-01T00:13:45.000,252
5480,21742445,2019-01-01T00:13:23.000,1524
5481,21742444,2019-01-01T00:08:13.000,4386


### Save Result Dataset

In [18]:
df_raw.to_csv(file_output_csv, index=False)
df_raw.to_parquet(file_output_parquet, index=False)